In [1]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
from datetime import date
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [37]:
# Using the code given, I have edited it to crawl in the twilight subreddit page from July 1 to July 30.
def to_utc(date):
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-07-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-07-30", "%Y-%m-%d")

#Create a range of dates to iterate 
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#prepare the parameters needed to call the API and look for 'twilight'
sort_type="score"
sort="desc"
fields=["author","subreddit","id","title","selftext","score","num_comments","created_utc"]
subreddit = 'twilight'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []

for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        
        time.sleep(1)

Doing 2020-07-01 to 2020-07-02
=====Done
Doing 2020-07-02 to 2020-07-03
=====Done
Doing 2020-07-03 to 2020-07-04
=====Done
Doing 2020-07-04 to 2020-07-05
=====Done
Doing 2020-07-05 to 2020-07-06
=====Done
Doing 2020-07-06 to 2020-07-07
=====Done
Doing 2020-07-07 to 2020-07-08
=====Done
Doing 2020-07-08 to 2020-07-09
=====Done
Doing 2020-07-09 to 2020-07-10
=====Done
Doing 2020-07-10 to 2020-07-11
=====Done
Doing 2020-07-11 to 2020-07-12
=====Done
Doing 2020-07-12 to 2020-07-13
=====Done
Doing 2020-07-13 to 2020-07-14
=====Done
Doing 2020-07-14 to 2020-07-15
=====Done
Doing 2020-07-15 to 2020-07-16
=====Done
Doing 2020-07-16 to 2020-07-17
=====Done
Doing 2020-07-17 to 2020-07-18
=====Done
Doing 2020-07-18 to 2020-07-19
=====Done
Doing 2020-07-19 to 2020-07-20
=====Done
Doing 2020-07-20 to 2020-07-21
=====Done
Doing 2020-07-21 to 2020-07-22
=====Done
Doing 2020-07-22 to 2020-07-23
=====Done
Doing 2020-07-23 to 2020-07-24
=====Done
Doing 2020-07-24 to 2020-07-25
=====Done
Doing 2020-07-25

In [38]:
results

[[{'author': 'I_late_sorry',
   'created_utc': 1593619366,
   'id': 'hjd3uf',
   'num_comments': 0,
   'score': 2,
   'selftext': '',
   'subreddit': 'twilight',
   'title': 'Delete scene from twilight https://vt.tiktok.com/UYQdTc/'},
  {'author': 'Daisylinne',
   'created_utc': 1593617946,
   'id': 'hjcmtm',
   'num_comments': 8,
   'score': 2,
   'selftext': '',
   'subreddit': 'twilight',
   'title': 'Everything is fun and games until you are Charlie Swan and all this crap happens to your daughter in less than two years'},
  {'author': 'giulia_gmnn',
   'created_utc': 1593614681,
   'id': 'hjbkaf',
   'num_comments': 14,
   'score': 2,
   'selftext': '',
   'subreddit': 'twilight',
   'title': 'my twilight obsession is out of hand, I bought Bellas clothes on ebay 😬'},
  {'author': 'wildflowergirl28',
   'created_utc': 1593588587,
   'id': 'hj5ogl',
   'num_comments': 3,
   'score': 1,
   'selftext': "Im reading twilight and Edward was telling Bella about carlisle's histories and she

In [39]:
flat_list = []
for sublist in results:
    if sublist is not None:
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)

#changing the date_utc to readable datetime and to readable day of the week

df['date'] = pd.to_datetime(df['created_utc'], unit='s').dt.strftime("%Y-%m-%d")
df['day'] = pd.to_datetime(df['created_utc'], unit ='ms').dt.strftime('%A')

#displaying first 5 entries
display(df.head())
df.to_csv("reddit_twilight.csv")

,author,created_utc,id,num_comments,score,selftext,subreddit,title,date,day
0,I_late_sorry,1593619366,hjd3uf,0,2,,twilight,Delete scene from twilight https://vt.tiktok.c...,2020-07-01,Monday
1,Daisylinne,1593617946,hjcmtm,8,2,,twilight,Everything is fun and games until you are Char...,2020-07-01,Monday
2,giulia_gmnn,1593614681,hjbkaf,14,2,,twilight,"my twilight obsession is out of hand, I bought...",2020-07-01,Monday
3,wildflowergirl28,1593588587,hj5ogl,3,1,Im reading twilight and Edward was telling Bel...,twilight,Can vampires talk to each other with speaking ...,2020-07-01,Monday
4,I_late_sorry,1593585905,hj56cd,9,1,Since Renesme is gonna live forever and Jacob ...,twilight,Is vampire werewolf hybrid possibles?,2020-07-01,Monday


**How many submissions were you able to gather?**
Since, each submission has the subreddit column filled out, i counted the number of rows in subreddit

In [40]:
total_sub = df['subreddit'].count()
print(total_sub)

747


**Who has the most submissions?**

In [41]:
most_sub = df['author'].mode()
print(most_sub)

0    I_late_sorry
dtype: object


**Which submission has the highest score?**
**Note**
code origin: https://stackoverflow.com/questions/15741759/find-maximum-value-of-a-column-and-return-the-corresponding-row-values-using-pan

In [42]:
print(df[df['score']==df['score'].max()])

           author  created_utc      id  num_comments  score selftext  \
390  I_late_sorry   1595095224  htkzg2            10    418            

    subreddit        title        date     day  
390  twilight  Poor Edward  2020-07-18  Monday  


**Which Submission has the highest number of comments?**

In [43]:
print(df[df['num_comments']==df['num_comments'].max()])

              author  created_utc      id  num_comments  score  \
691  paternalpadfoot   1596043519  i04muz           117      1   

                                              selftext subreddit  \
691  I have a huge list, but I’m always sad that we...  twilight   

                                                 title        date     day  
691  What book scene do you wish had made it in the...  2020-07-29  Monday  


**Which day of the week has the most submissions?**

In [44]:
day_of_week = df['day'].mode()
print(day_of_week)

0    Monday
dtype: object
